### Recurrent Neural Networks - Assignment
#### by Gaurav Singh (grv08singh@gmail.com)

In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from keras.datasets import imdb
#from keras.utils import pad_sequences
from keras.preprocessing.sequence import pad_sequences
from keras import Sequential
from keras.layers import Dense, SimpleRNN, Embedding


import warnings as wr
wr.filterwarnings('ignore')

#### Problem Statement:
Build a sentiment analysis model using `Recurrent Neural Networks (RNNs)` to classify movie reviews from the IMDB dataset into __positive__ or __negative__ sentiments.

##### Dataset:
The dataset comprises __25,000 movie reviews__ from `IMDB`, labeled by sentiment (`positive`/`negative`). Reviews have been preprocessed, and each review is encoded as a sequence of word indices (integers). The words in the dataset are indexed by overall frequency in the dataset, allowing for quick filtering operations such as: "only consider the top 10,000 most common words, but eliminate the top 20 most common words".

#### Tasks to be Performed:

##### 1) Data Preprocessing:
* __Load__ the IMDB dataset, keeping only the __top 10,000__ most frequently occurring words.

In [2]:
vocab_cap = 10000
(X_train, y_train),(X_test, y_test) = imdb.load_data(num_words=vocab_cap)

* __Pad__ the sequences so that they all have the same length.

In [3]:
#Average length per review
review_len = []
for review in X_train:
    review_len.append(len(review))
print(f"Minimum review length : {np.min(review_len)}")
print(f"Maximum review length : {np.max(review_len)}")
print(f"Avg review length : {np.average(review_len)}")

Minimum review length : 11
Maximum review length : 2494
Avg review length : 238.71364


In [4]:
#keeping only first 500 words
max_len = 500
X_train_pad = pad_sequences(X_train, maxlen=max_len)
X_test_pad = pad_sequences(X_test, maxlen=max_len)

##### 2) Model Building:
* __Create__ a Sequential RNN model using `TensorFlow` and `Keras`.
* The model should consist of an `Embedding` layer, a `SimpleRNN` layer, and a `Dense` output layer.

In [5]:
model = Sequential()
model.add(Embedding(input_dim=vocab_cap, output_dim=32, input_length=max_len))
model.add(SimpleRNN(64, return_sequences=False, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

#input_dim: Size of the vocabulary
#output_dim: Size of the output of embedding layer
#input_length: Size of each review (number of words in each review.)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

* __Compile__ the model, specifying the appropriate `optimizer`, `loss` function, and `metrics`.

In [6]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

##### 3) Training:
* __Train__ the model on the preprocessed movie reviews, using a `batch size` of __128__ and `validating` on __20%__ of the `training data`.
* __Run__ the training for `10 epochs`.

In [7]:
history = model.fit(
    X_train_pad,
    y_train,
    batch_size=128,
    epochs=10,
    validation_split=0.2
)

Epoch 1/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 16s 90ms/step - accuracy: 0.5573 - loss: 13990352.0000 - val_accuracy: 0.5062 - val_loss: 2386.3560
Epoch 2/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 14s 89ms/step - accuracy: 0.6368 - loss: 132.7913 - val_accuracy: 0.6442 - val_loss: 0.6376
Epoch 3/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 14s 89ms/step - accuracy: 0.6932 - loss: 0.6093 - val_accuracy: 0.6832 - val_loss: 0.6061
Epoch 4/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 15s 92ms/step - accuracy: 0.7046 - loss: 0.6028 - val_accuracy: 0.6780 - val_loss: 0.6161
Epoch 5/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 14s 87ms/step - accuracy: 0.7239 - loss: 0.5750 - val_accuracy: 0.7118 - val_loss: 0.5791
Epoch 6/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 12s 79ms/step - accuracy: 0.7545 - loss: 0.5355 - val_accuracy: 0.7224 - val_loss: 0.6068
Epoch 7/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 12s 76ms/step - accuracy: 0.7727 - loss: 0.5112 - val_accuracy: 0.7420 - val_loss: 0.5475
Epoch 8/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 21s 79ms/step - accuracy: 0.7901 - los

##### 4) Evaluation:
* __Evaluate__ the model on the `test` set and __report__ the `accuracy`.

In [8]:
test_loss, test_accuracy = model.evaluate(x=X_test_pad, y=y_test, batch_size=128)
print(f"Test Accuracy: {test_accuracy}")

196/196 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - accuracy: 0.7464 - loss: 0.5342
Test Accuracy: 0.7463600039482117


* __Expected Outcome__:
A trained `RNN model` that can classify movie reviews into `positive` or `negative` sentiments, with an `accuracy metric` provided at the end of the training process.

In [10]:
#predicting first n reviews from test dataset
n = 100
pred = model.predict(X_test_pad[:n])
for i in range(n):
    if i%5 == 0:
        print()
    if pred[i][0] > 0.5:
        confidence = pred[i][0]
        pred_sentiment = 'Positive'
    else:
        confidence = 1 - pred[i][0]
        pred_sentiment = 'Negative'
    actual = 'Positive' if y_test[i] == 1 else 'Negative'
    mark = '✓' if pred_sentiment==actual else '✗'
    print(f"Confidence Level : {100*confidence:.0f}% | Predicted Sentiment : {pred_sentiment} | Actual Sentiment : {actual} | {mark}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step

Confidence Level : 73% | Predicted Sentiment : Negative | Actual Sentiment : Negative | ✓
Confidence Level : 98% | Predicted Sentiment : Positive | Actual Sentiment : Positive | ✓
Confidence Level : 62% | Predicted Sentiment : Negative | Actual Sentiment : Positive | ✗
Confidence Level : 69% | Predicted Sentiment : Negative | Actual Sentiment : Negative | ✓
Confidence Level : 55% | Predicted Sentiment : Positive | Actual Sentiment : Positive | ✓

Confidence Level : 75% | Predicted Sentiment : Negative | Actual Sentiment : Positive | ✗
Confidence Level : 90% | Predicted Sentiment : Positive | Actual Sentiment : Positive | ✓
Confidence Level : 70% | Predicted Sentiment : Negative | Actual Sentiment : Negative | ✓
Confidence Level : 82% | Predicted Sentiment : Positive | Actual Sentiment : Negative | ✗
Confidence Level : 93% | Predicted Sentiment : Positive | Actual Sentiment : Positive | ✓

Confidence Level : 82% | Predicted Sentiment : Positive | A